In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# def fix_address(address):
#     return (address.split(sep=', '))[0]

In [ ]:
# conda install -c conda-forge geopy
from geopy.geocoders import Nominatim
from geopy.distance import distance
from tqdm import tqdm

# proceed with cauction!!!
def distance_from(given_location, dataframe, addresses_column_name, postifx="", inplace=False):
    """
    dataframe - pandas.DataFrame object
    addresses_column_name - name of the column in which we can find addresses
    given_location - tuple with latitude and longitude of a location from which we're counting the distance

    postfix - what should I add to every address to make sure it's what we need. example: city/country
    inplace - check it if you want to change the given dataframe. default False

    returned value - pd.Series distances in kilometers
    """
    geolocator = Nominatim(user_agent="Strive School YMVC Project")

    number_of_rows = dataframe.shape[0]
    addresses_df = dataframe[addresses_column_name]

    gps_addresses = np.zeros(shape=number_of_rows)

    # this loop can take a looooong time.
    for row_nr in tqdm(np.arange(0, number_of_rows)):
        dist = np.nan
        try:
            loc = geolocator.geocode(addresses_df.iloc[row_nr] + postifx)
            dist = distance(given_location, (loc.latitude, loc.longitude)).kilometers
        except Exception as e:
            print(e)
        gps_addresses[row_nr] = dist

    if inplace:
        # we put a new column before the "address" column
        dataframe.insert(dataframe.columns.get_loc(addresses_column_name), 'distance_km', gps_addresses)
    else:
        return pd.Series(gps_addresses, name='distance_km')

In [ ]:
def contains(word, column):
    """
    Usage:
    Pass a column and a word u want to find in the cells, to get a np.array of type bool.
    Afterwards u can pass it as an argument of [] selection mechanism. Examples:

    austin_tx[ contains(austin_tx.categories, "Shopping") ]
    austin_tx[ contains(austin_tx.categories, "Shopping") | contains(austin_tx.categories, "Hotels") ]      # and
    austin_tx[ contains(austin_tx.categories, "Shopping") & contains(austin_tx.categories, "Watches") ]     # or

    Works for all columns that return strings. hours and attributes too.
    """
    return np.array([(word in vals) for vals in column])

In [ ]:
# def fix_category(category):
#     return category.replace(", &", " &")

In [ ]:
def get_categories_dict(category_column, separator):
    categories_counts = {}
    for things in category_column:
        for thing in things.split(sep=separator):    
            if thing in categories_counts.keys():
                categories_counts[thing] += 1
            else:
                categories_counts[thing] = 1
    return categories_counts

In [ ]:
berlin = pd.read_csv('yelp_dataset.csv')

In [ ]:
berlin.drop(['url', 'claimed'], axis=1, inplace=True)

In [ ]:
print(berlin.columns.values)

In [ ]:
berlin.isna().sum()

In [ ]:
berlin.drop(berlin.index[berlin['attributes'].isna() | berlin['categories'].isna()], inplace=True)


In [ ]:
berlin.isna().sum()

In [ ]:
# import re
def fix_names(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].apply(lambda name: e.sub(r"(\w)([A-Z])", r"\1 \2", name))

In [ ]:
# fix_names(berlin, 'Restaurants_Name')

In [ ]:
berlin.shape[0]

In [ ]:
# graph stars vs distance

In [ ]:
distance_from((52.525579992852855, 13.369230338630214),berlin, 'address',  inplace=True)
# berlin.drop(berlin['distance_km'].isna())

In [ ]:
berlin.dropna(inplace=True)

In [ ]:
categories = get_categories_dict(berlin.categories, ',')            # dictionary of categories
cat_df = pd.DataFrame([categories.keys(), categories.values()]).T   # dataframe out of that dictionary
cat_df.rename(columns={0:'category',1:'count'}, inplace=True)       # name columns in a fasionable name

In [ ]:
avg_stars=[]
avg_rev_count=[]
for category in cat_df['category']:
    avg_stars.append(berlin[contains(category, berlin.categories)]['stars'].mean())
    avg_rev_count.append(berlin[contains(category, berlin.categories)]['review_count'].mean())

cat_df['average_stars'] = avg_stars
cat_df['average_review_count'] = avg_rev_count

In [ ]:
cat_df.sort_values(by='count', ascending=False).head(20)